In [1]:
!pip install tensorflow

In [2]:
!pip install scikeras

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("data/all/meteo/sampled_df_10000.csv")

In [3]:
df.drop(['date', 'geometry', 'TIPOBOSQUE', 'UsoIFN', 'n_CLAMFE', 'UsoMFE'], axis=1, inplace=True)
df

,altitud,pendiente,orientacion,is_fire,n_CLAIFN,t_max,u,v,specific_humidity,relative_humidity
0,226.43176,1.512323,287.237460,False,200,302.235107,-0.482693,-1.057308,0.011643,51.497208
1,591.98520,37.829388,174.613740,True,141,291.587646,5.145885,-2.061542,0.002844,29.498014
2,302.96150,57.214653,76.929750,True,111,306.062988,0.700964,0.509414,0.006809,24.005035
3,365.60150,36.542430,112.896065,False,141,298.082031,-2.383409,-1.082997,0.013121,67.682327
4,216.34225,48.399876,129.588600,True,140,305.471191,4.591745,-0.290295,0.004589,14.796286
...,...,...,...,...,...,...,...,...,...,...
9995,321.46176,58.597137,240.593480,False,121,300.824463,-1.734298,0.723990,0.010968,48.677845
9996,998.84950,23.794138,231.235290,True,171,302.501221,-1.233615,-0.148222,0.009740,39.612019
9997,79.08550,39.971180,339.268550,False,300,280.333252,-1.505415,-3.901253,0.003711,66.535889
9998,288.20276,34.997368,306.008120,False,111,291.637451,1.306460,0.193386,0.004443,35.163040


In [4]:
df.describe()

,altitud,pendiente,orientacion,n_CLAIFN,t_max,u,v,specific_humidity,relative_humidity
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,446.083088,28.564042,170.111652,162.562600,299.449704,0.833451,0.029887,0.006179,35.056203
std,317.022152,26.731821,102.814267,56.652146,8.523888,1.992910,1.443016,0.002585,17.824743
min,0.000002,0.000000,0.000000,111.000000,278.099609,-9.098049,-11.459216,0.000565,2.928324
25%,179.453750,6.494645,83.989180,121.000000,292.094604,-0.495015,-0.523342,0.004366,21.858106
50%,419.941000,23.883654,161.301620,141.000000,299.799561,0.646391,0.244891,0.005690,33.624155
75%,680.122500,44.726578,254.606018,200.000000,305.702881,1.929230,0.790262,0.007353,46.508265
max,1627.506200,475.853200,359.857670,500.000000,315.612061,11.758718,6.209616,0.016309,95.826477


In [5]:
# Select quantitatives variables
cuant_var = df[["altitud", "pendiente", "orientacion", "t_max", "u", "v", "specific_humidity", "relative_humidity"]]
# Normalize
mean = cuant_var.mean()
mean.to_csv("data/nn/means.csv")
std = cuant_var.std()
std.to_csv("data/nn/stds.csv")
normalized_df = (cuant_var-cuant_var.mean())/cuant_var.std()
# Add soil use binarizaded
normalized_df = normalized_df.join(pd.get_dummies(df["n_CLAIFN"].astype("category")))

In [6]:
# Split dataset
y = df["is_fire"]
xTrain, xTest, yTrain, yTest = train_test_split(normalized_df, y, test_size=0.2, random_state=12)

In [15]:
xTest

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity,111,112,...,141,142,150,161,171,172,200,300,400,500
5669,0.791761,0.281929,-0.155047,0.570548,-0.461864,0.027699,0.485999,0.015338,1,0,...,0,0,0,0,0,0,0,0,0,0
8800,1.080497,-0.713308,-1.345743,0.026523,-0.910445,-0.049421,1.275303,1.257824,0,0,...,1,0,0,0,0,0,0,0,0,0
3205,0.769821,0.982115,-1.592882,-0.145930,-1.630387,-1.061468,2.811011,1.934182,0,0,...,1,0,0,0,0,0,0,0,0,0
8731,0.395582,-0.511498,0.443695,1.406692,0.616303,0.241257,-0.415831,-1.431236,1,0,...,0,0,0,0,0,0,0,0,0,0
6412,1.695439,1.466722,0.173093,-0.481614,-0.994136,-0.547441,0.463707,0.825513,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,-1.362713,-0.692165,-1.648495,-1.257524,-1.178980,-0.579903,-1.169310,-0.317053,0,0,...,0,0,0,0,0,0,0,1,0,0
142,0.931766,-0.062998,1.105972,-1.612311,1.982156,-2.971841,-1.622293,-0.551655,0,0,...,0,0,0,0,0,0,0,0,0,0
275,-0.889621,1.199585,-1.482392,-1.238219,-1.543729,-1.785804,0.126060,1.526523,0,0,...,0,0,0,0,0,0,0,1,0,0
8265,0.497148,0.318305,-0.947187,0.630667,-0.403922,0.123957,0.285282,-0.151447,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
xTest.columns

Index([          'altitud',         'pendiente',       'orientacion',
                   't_max',                 'u',                 'v',
       'specific_humidity', 'relative_humidity',                 111,
                       112,                 114,                 121,
                       122,                 124,                 132,
                       140,                 141,                 142,
                       150,                 161,                 171,
                       172,                 200,                 300,
                       400,                 500],
      dtype='object')

In [7]:
len(xTest.columns)

26

In [8]:
yTest

5669     True
8800     True
3205     True
8731     True
6412    False
        ...  
794     False
142     False
275     False
8265     True
7950     True
Name: is_fire, Length: 2000, dtype: bool

In [19]:
# Create function returning a model
def design_model (dropout_rate, weight_constraint):

    model = keras.Sequential()

    model.add(keras.layers.Dense(64, input_shape=(26,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(weight_constraint)))
    model.add(keras.layers.Dropout(dropout_rate))
    
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model

model = KerasClassifier(model=design_model, verbose=0)

In [11]:
# Define the search parameters
epochs = [10,20,50,100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [ 0.2, 0.3, 0.4, 0.5]

In [12]:
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint, optimizer=optimizer, epochs=epochs)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [11]:
grid_result = grid.fit(xTrain, yTrain)

PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
# Print results
print(f'Best:{grid_result.best_score_} using {grid_result.best_params_}')

# Best:0.9348733415074802 using {'epochs': 100, 'model__dropout_rate': 0.2, 'model__weight_constraint': 3.0, 'optimizer': 'Nadam'}

for mean, stdev, param in zip(grid_result.cv_results_['mean_test_score'], 
                              grid_result.cv_results_['std_test_score'], grid_result.cv_results_['params']):
    print(f"{mean} {stdev} with: {param}")


In [9]:
##### Design final model with best parameters
model = keras.Sequential()

model.add(keras.layers.Dense(64, input_shape=(26,), activation='relu', kernel_constraint=keras.constraints.MaxNorm(3)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(256, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(128, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(64, activation='relu', kernel_constraint=keras.constraints.MaxNorm(1)))
model.add(keras.layers.Dropout(0.2))
    
model.add(keras.layers.Dense(2, activation='softmax'))

In [10]:
# Define the accuracy metrics and parameters
model.compile(optimizer="Nadam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3183 - accuracy: 0.8675
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2596 - accuracy: 0.9006
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2430 - accuracy: 0.9100
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2327 - accuracy: 0.9149
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2240 - accuracy: 0.9144
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2186 - accuracy: 0.9168
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.2143 - accuracy: 0.9165
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2053 - accuracy: 0.9208
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2069 - accuracy: 0.9211
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2041 - accura

In [12]:
# Export model
model.save("data/nn/nn_fire_risk.h5")

In [22]:
# Load model
model = keras.models.load_model('data/nn/nn_fire_risk_all_26.h5')

In [13]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("\nConfusion matrix:", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

63/63 [==============================] - 0s 558us/step

Confusion matrix: [[933  68]
 [ 45 954]]

P-Score: 0.933, R-Score: 0.955


In [14]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(xTest, yTest)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 0s 544us/step - loss: 0.1844 - accuracy: 0.9435
Test Loss: 0.1844322234392166
Test Accuracy: 0.9434999823570251


In [14]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 5.0 MB/s eta 0:00:00
  Installing build depen

In [ ]:
import tensorflowjs as tfjs

# Convert the Keras model to TensorFlow.js format
tfjs.converters.save_keras_model(model, '/data/nn_fire_risk_js')

In [27]:
xTest_list_1 = [546.37,21.421358,80.22286,300.30322265625,4.351698398590088,6.368526458740234,0.005158774554729462,27.214874267578125]
xTest_1 = pd.DataFrame([xTest_list_1])
xTest_1

,0,1,2,3,4,5,6,7
0,546.37,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874


In [20]:
type(xTest)

pandas.core.frame.DataFrame

In [33]:
xTest_list_1 = [546.37,21.421358,80.22286,300.30322265625,4.351698398590088,6.368526458740234,0.005158774554729462,27.214874267578125]
xTest_1 = pd.DataFrame([xTest_list_1])
yTest_1_predicted = model.predict(xTest_1)
yTest_1_predicted

1/1 [==============================] - 0s 16ms/step


array([[0., 1.]], dtype=float32)

In [37]:
xTest_1

,0,1,2,3,4,5,6,7
0,546.37,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874


In [35]:
yTest_1_predicted

array([[0., 1.]], dtype=float32)

In [34]:
xTest_list_2 = [10.5720005,2.4017253,330.45306,295.8662109375,-0.29025888442993164,-4.314094543457031,0.011048879474401474,65.92393493652344]
xTest_2 = pd.DataFrame([xTest_list_2])
yTest_2_predicted = model.predict(xTest_2)
yTest_2_predicted

1/1 [==============================] - 0s 12ms/step


array([[0., 1.]], dtype=float32)

In [40]:
xTest_list_3 = [xTest_list_1, xTest_list_2]
xTest_list_3

[[546.37,
  21.421358,
  80.22286,
  300.30322265625,
  4.351698398590088,
  6.368526458740234,
  0.005158774554729462,
  27.214874267578125],
 [10.5720005,
  2.4017253,
  330.45306,
  295.8662109375,
  -0.29025888442993164,
  -4.314094543457031,
  0.011048879474401474,
  65.92393493652344]]

In [42]:
xTest_3 = pd.DataFrame(xTest_list_3)
xTest_3

,0,1,2,3,4,5,6,7
0,546.370,21.421358,80.22286,300.303223,4.351698,6.368526,0.005159,27.214874
1,10.572,2.401725,330.45306,295.866211,-0.290259,-4.314095,0.011049,65.923935


In [46]:
xTest_3 = xTest.copy()

In [49]:
xTest_3.iloc[0]

altitud             -0.293634
pendiente            0.165739
orientacion          1.406116
t_max               -0.119670
u                    1.500631
v                    0.522767
specific_humidity    0.107386
relative_humidity    0.694872
Name: 518, dtype: float64

In [56]:
xTest_3.head(3)

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity
518,-0.293634,0.165739,1.406116,-0.119670,1.500631,0.522767,0.107386,0.694872
871,-0.954662,-0.827133,0.010550,-1.204439,0.311809,-0.834331,-0.903463,-0.018830
797,-0.576325,-0.832027,-0.043692,-1.091953,-0.654979,-0.306512,-0.398518,0.588766


In [69]:
xTest_3 = xTest.head(100)

In [70]:
xTest_3

,altitud,pendiente,orientacion,t_max,u,v,specific_humidity,relative_humidity
518,-0.293634,0.165739,1.406116,-0.119670,1.500631,0.522767,0.107386,0.694872
871,-0.954662,-0.827133,0.010550,-1.204439,0.311809,-0.834331,-0.903463,-0.018830
797,-0.576325,-0.832027,-0.043692,-1.091953,-0.654979,-0.306512,-0.398518,0.588766
274,-0.956474,-0.881371,0.507296,-1.250444,0.438949,-2.396988,-0.962472,-0.228474
325,-0.031658,1.800683,-0.880576,0.757039,-0.164712,-0.664707,1.873592,1.023697
...,...,...,...,...,...,...,...,...
403,1.438310,1.671688,-0.820983,1.381804,0.563844,1.368574,-0.874234,-1.824139
886,-0.938628,-0.870692,-0.881941,-0.174605,0.231181,-0.121332,-0.753221,-1.031635
542,-0.956001,-0.850536,0.793985,-0.494520,-0.974360,1.239939,0.257952,1.031888
29,-0.778937,-0.648503,0.135234,0.407176,-0.462611,-0.320644,1.506346,1.109932


In [73]:
yTest_3_predicted = model.predict(xTest_3)
results_3 = (yTest_3_predicted[:,1]>0.5).astype(int)

4/4 [==============================] - 0s 1ms/step


In [74]:
results_3

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0])

In [77]:
np.where(results_3 == 1)

(array([27, 35, 45, 48, 71, 76, 81, 83, 86, 87, 88, 90, 91, 95]),)

In [80]:
xTest_3.iloc[27]

altitud              1.068535
pendiente            0.179948
orientacion          0.354402
t_max                1.389336
u                    0.554170
v                    1.370858
specific_humidity   -0.890492
relative_humidity   -1.837286
Name: 488, dtype: float64